In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with PyTorch on Ray on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/get_started_with_pytorch_rov.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fray_on_vertex_ai%2Fget_started_with_pytorch_rov.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/get_started_with_pytorch_rov.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/get_started_with_pytorch_rov.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This tutorial demonstrates how to use Ray on Vertex AI SDK and Vertex AI SDK for training and serving an PyTorch image classification model.

Learn more about [Ray on Vertex AI overview](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview).

### Objective

In this tutorial, you learn how to efficiently distribute the training process of a PyTorch image classification model by leveraging Ray on Vertex AI. Furthermore, you learn how to deploy the trained model seamlessly to Vertex AI Endpoint.

This tutorial uses the following Google Cloud ML services and resources:

- Ray on Vertex AI
- Vertex AI Model Registry
- Vertex AI Prediction


The steps performed include:

- Prepare the training script
- Submit a Ray job using the Ray Jobs API
- Download a trained image model from PyTorch
- Create a custom model handler
- Package model artifacts in a model archive file
- Register model in Vertex AI Model Registry
- Deploy model in Vertex AI Endpoint
- Make online predictions

### Dataset

This tutorial uses the [CIFAR-10 dataset](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html) which consists of 60000 32x32 colour images in 10 classes, with 6000 images per class.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""

! pip3 install {USER} ipywidgets==8 fsspec==2023.12.2 google-cloud-aiplatform[ray]==1.40.0 ray[data]==2.4.0 ray[train]==2.4.0 ray[tune]==2.4.0 -q --no-warn-conflicts
! pip3 install {USER} torch==2.0.0 torchvision==0.15.1 torchmetrics==0.11.4 torchserve==0.7.1 torch-model-archiver==0.7.1 -q --no-warn-conflicts
! pip3 install {USER} google-auth==2.27.0 etils==1.5.2 -q --no-warn-conflicts

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Add torch-model-archiver to the PATH

Update the `PATH` environment variable to add `torch-model-archiver`

In [ ]:
import os

os.environ["PATH"] = f'{os.environ.get("PATH")}:~/.local/bin'

### Set a Ray on Vertex AI cluster

Before to run the code below, make sure to [set up](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/set-up) Ray on Vertex AI and [create](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/create-cluster) at least one Ray on Vertex AI cluster.

In [ ]:
import vertex_ray
from google.cloud import aiplatform as vertex_ai
from vertex_ray import Resources

#### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### Define cluster configuration

To provision a Ray cluster on Vertex AI, you can use a default provisioning request or you can specify the replica count (number of nodes), machine type, disk_spec, and accelerator as needed.

In [ ]:
head_node_type = Resources(
    machine_type="n1-standard-8",
    node_count=1,
)

worker_node_types = [
    Resources(
        machine_type="n1-standard-8",
        node_count=2,
    )
]

#### Create the Ray cluster

Create the Ray cluster using the Ray on Vertex AI Python SDK.

In [ ]:
cluster_name = f"ray-cluster-{TIMESTAMP}"

In [ ]:
ray_cluster_name = vertex_ray.create_ray_cluster(
    head_node_type=head_node_type,
    worker_node_types=worker_node_types,
    cluster_name=cluster_name,
)

#### Get the Ray cluster

Use the Ray on Vertex AI Python SDK to get the Ray cluster.

In [ ]:
ray_clusters = vertex_ray.list_ray_clusters()
ray_cluster_resource_name = ray_clusters[-1].cluster_resource_name
ray_cluster = vertex_ray.get_ray_cluster(ray_cluster_resource_name)

In [ ]:
print("Ray cluster on Vertex AI:", ray_cluster_resource_name)

### Set tutorial folder

Set up the folder you use in this tutorial.

In [ ]:
from pathlib import Path as path

root_path = path.cwd()
tutorial_path = root_path / "tutorial"
src_path = tutorial_path / "src"
deliverables_path = tutorial_path / "deliverables"
build_path = tutorial_path / "build"
tests_path = tutorial_path / "tests"

src_path.mkdir(parents=True, exist_ok=True)
deliverables_path.mkdir(parents=True, exist_ok=True)
build_path.mkdir(parents=True, exist_ok=True)
tests_path.mkdir(parents=True, exist_ok=True)

### Import libraries

In [ ]:
import base64
import io
import os
import random
import shutil
import string
import time

# Ray - Training
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
# General
from etils import epath
from matplotlib import pyplot as plt
from ray.job_submission import JobStatus, JobSubmissionClient
# Serving
from ray.tune import ExperimentAnalysis
from vertex_ray.predict import torch as ray_torch

### Set variables

In [ ]:
# training
LOGGING_URI = epath.Path(BUCKET_URI) / "logs"
EXPERIMENT_NAME = "torch_on_rov"
TRAINING_URI = LOGGING_URI / EXPERIMENT_NAME

# serving
DELIVERABLES_PATH = str(deliverables_path)
BUILD_URI = str(epath.Path(BUCKET_URI) / "build")
DEPLOY_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest"
MODEL_NAME = "torch_on_rov_cifar10"
DEPLOY_COMPUTE = "n1-standard-4"

### Define helpers

In [ ]:
def plot_image_sample(test_dataset):
    """Plots a sample image from the CIFAR-10 dataset."""

    sample_idx = random.randrange(0, len(test_dataset))
    image, _ = test_dataset[sample_idx]
    pil_image = transforms.ToPILImage()(image)
    plt.imshow(pil_image)
    plt.show()

    return pil_image


def predict_from_image(pil_image, endpoint):
    """Predicts the class of an image using the given endpoint."""
    buffered_image = io.BytesIO()
    pil_image.save(buffered_image, format="JPEG")

    data = {"data": base64.b64encode(buffered_image.getvalue()).decode("utf-8")}
    response = endpoint.predict(instances=[data])

    return response.predictions

## Training a PyTorch model

In this tutorial, you train a custom image classification model using Ray on Vertex AI.

#### Prepare the training application

Before you start the training, let's take a look at how Ray job might be assembled to distribute your training.

Ray 2.4.0 uses `train_loop_per_worker` fuction to a distributed multi-worker training function.

After you set up your dataset and your model, you define your single-worker PyTorch training function and then you convert it to distributed multi-worker training function as followed:

1. Use the `ray.train.torch.prepare_data_loader` to wrap your data with  `DistributedSampler` for distributed training.

2. Use the `ray.train.torch.prepare_model` function to wrap your model with  `DistributedDataParallel` for distributed training.

After you have your multi-worker training function, you need to define the `ScalingConfig` to specify the desired number of workers and indicate whether the distributed training process requires GPUs.

Additionally, you can define a `RunConfig` to specify checkpointing and synchronization behaviors along the distributed training workload and some additional training loop parameters.

Finally, you pass everything to `TorchTrainer` which Ray uses to distribute your training utilizing Distributed Data Parallelism (using PyTorch’s Distributed backend).

### Prepare the training script

The file `src/task.py` is the Python script for executing the Ray distributed training job.

In [ ]:
training_script = """
# libraries

# general libraries
import os
import argparse
from etils import epath

# training libraries
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchmetrics.aggregation import MeanMetric
from torchmetrics.classification.accuracy import Accuracy

# ray libraries
import ray
from ray import train
from ray.train.torch import TorchTrainer, TorchCheckpoint
from ray.tune import ExperimentAnalysis
from ray.tune.syncer import SyncConfig
from ray.air import session, Checkpoint, CheckpointConfig
from ray.air.config import ScalingConfig, RunConfig


# helpers
def get_args():
    parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
    parser.add_argument('--batch-size', dest='batch_size',
                        type=int, default=16, help='Batch size')
    parser.add_argument('--epochs', dest='epochs',
                        type=int, default=10, help='Number of epochs')
    parser.add_argument('--lr', dest='lr',
                        type=int, default=1e-3, help='Learning rate')
    parser.add_argument('--num-workers', dest='num_workers',
                        type=int, default=1, help='Number of workers')
    parser.add_argument('--use-gpu', dest='use_gpu', action='store_true',
                        default=False, help='Use GPU')
    parser.add_argument('--experiment-name', dest='experiment_name', type=str,
                        default='cifar10-torch', help='Experiment name')
    parser.add_argument('--logging-dir', dest='logging_dir',
                        type=str, default='./logs', help='Logging directory')
    args = parser.parse_args()
    return args


# Create a simple model
class Cifar10Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def train_epoch(train_loader, model, loss_fn, optimizer, device):
    # initiate training
    model.train()
    train_loss = MeanMetric()
    train_accuracy = Accuracy(task="multiclass", num_classes=10)

    # load metrics to device
    train_loss.to(device)
    train_accuracy.to(device)

    # training loop
    for batch, (data, target) in enumerate(train_loader):
        # move data to device
        data = data.to(device)
        target = target.to(device)

        # compute error
        output = model(data)
        loss = loss_fn(output, target)
        train_loss.update(loss)
        train_accuracy.update(output, target)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print results
        if batch % 100 == 0:
            print(f'batch {batch}, loss {loss.item():.4f}')

    # compute loss and metrics
    train_loss = Tensor.numpy(train_loss.compute(), force=True).item()
    train_accuracy = Tensor.numpy(train_accuracy.compute(), force=True).item()

    return train_loss, train_accuracy


def evaluate_epoch(test_loader, model, loss_fn, device):
    # initiate evaluation
    model.eval()
    test_loss = MeanMetric()
    test_accuracy = Accuracy(task="multiclass", num_classes=10)

    # load metrics to device
    test_loss.to(device)
    test_accuracy.to(device)

    # evaluation loop
    for batch, (data, target) in enumerate(test_loader):
        with torch.no_grad():
            # move data to device
            data = data.to(device)
            target = target.to(device)

            # get loss and accuracy
            output = model(data)
            test_loss.update(loss_fn(output, target))
            test_accuracy.update(output, target)

    # compute loss and metrics
    test_loss = Tensor.numpy(test_loss.compute(), force=True).item()
    test_accuracy = Tensor.numpy(test_accuracy.compute(), force=True).item()

    return test_loss, test_accuracy


def train_loop_per_worker(config):
    # set configuration
    batch_size = config["batch_size"]
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]

    # get device
    device = train.torch.get_device() if torch.cuda.is_available() else torch.device("cpu")

    # read dataset
    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    transform = transforms.Compose([transforms.ToTensor(), normalize])

    train_dataset = datasets.CIFAR10(root="./train",
                                     transform=transform,
                                     train=True, download=True)

    test_dataset = datasets.CIFAR10(root="./test",
                                    transform=transform,
                                    train=False, download=True)

    # create data loaders
    train_loader = data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = data.DataLoader(test_dataset, batch_size=batch_size)
    train_loader = train.torch.prepare_data_loader(train_loader)
    test_loader = train.torch.prepare_data_loader(test_loader)

    # create model
    model = Cifar10Model()
    model = train.torch.prepare_model(model)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # train model
    for epoch in range(1, epochs + 1):
        train_loss, train_accuracy = train_epoch(train_loader, model, loss_fn, optimizer, device)
        test_loss, test_accuracy = evaluate_epoch(test_loader, model, loss_fn, device)

        # report metrics and model checkpoint
        session.report(
            metrics={"train_loss": train_loss, "train_accuracy": train_accuracy,
                     "test_loss": test_loss, "test_accuracy": test_accuracy},
            checkpoint=TorchCheckpoint.from_model(model)
        )


def main():
    # set configuration
    args = get_args()
    config = vars(args)

    # initialize ray session
    ray.init()

    # train model
    train_loop_config = {"learning_rate": config['lr'], "batch_size": config['batch_size'],
                         "epochs": config['epochs']}
    scaling_config = ScalingConfig(num_workers=config['num_workers'], use_gpu=config['use_gpu'])
    run_config = RunConfig(checkpoint_config=CheckpointConfig(num_to_keep=1),
                           sync_config=SyncConfig(upload_dir=config['logging_dir']),
                           name=config['experiment_name'])

    trainer = TorchTrainer(
        train_loop_per_worker=train_loop_per_worker,
        train_loop_config=train_loop_config,
        run_config=run_config,
        scaling_config=scaling_config
    )
    result = trainer.fit()
    print(f"Last result: {result.metrics}")


if __name__ == "__main__":
    main()
"""

with open(src_path / "task.py", "w") as f:
    f.write(training_script)
f.close()

### Prepare the `requirements` file

The file `requirements.txt` includes the dependencies your Ray application needs to run.

In [ ]:
requirements = """
ipywidgets==8
fsspec==2023.12.2
etils==1.5.2
importlib-resources==6.1.1
google-cloud-aiplatform[ray]==1.40.0
ray[data]==2.4.0
ray[train]==2.4.0
ray[tune]==2.4.0
torch==2.0.0
torchvision==0.15.1
torchmetrics==0.11.4
"""

with open(tutorial_path / "requirements.txt", "w") as f:
    f.write(requirements)
f.close()

### Submit a Ray job using the Ray Jobs API

Submit the script to the Ray on Vertex AI cluster using the Ray Jobs API with  the public Ray dashboard address.

It is important to highlight that Ray Jobs API is the prefered option if you would rather submit jobs programmatically. You can also use the Ray on Vertex AI SDK if you prefer an interactive Python development environment.

Initiate the client to submit the job.

In [ ]:
client = JobSubmissionClient(
    address="vertex_ray://{}".format(ray_cluster.dashboard_address)
)

Submit the job.

In [ ]:
id = "".join(random.choices(string.ascii_lowercase + string.digits, k=4))

job_id = client.submit_job(
    submission_id=f"ray-job-{TIMESTAMP}-{id}",
    entrypoint=f"python3 task.py --experiment-name={EXPERIMENT_NAME} --num-workers=2 --logging-dir={LOGGING_URI}",
    runtime_env={
        "pip": {"packages": str(tutorial_path / "requirements.txt")},
        "working_dir": str(src_path),
    },
)

Check the status of the job.

In [ ]:
while True:
    job_status = client.get_job_status(job_id)
    if job_status == JobStatus.SUCCEEDED:
        print("Job succeeded!")
        break
    else:
        if job_status == JobStatus.FAILED:
            print("Job failed!")
            break
        else:
            print("Job is running...")
            time.sleep(60)

### Check model artifacts

When the Ray training job has completed, you check the model artifacts in the Cloud Storage location.


In [ ]:
! gsutil ls -l {TRAINING_URI}

## Serving a PyTorch model

You can serve a PyTorch model on Vertex AI using TorchServe as followed:

1. Download Ray training checkpoints.
2. Get PyTorch model from the Ray TorchCheckpoint.
3. Package the trained model artifacts including the model artifact, the model module and the custom handler by creating an archive file using the Torch Model Archiver tool.
4. Register model in Vertex AI Model Registry.
5. Deploy model to Vertex AI Endpoint for predictions.

### Download Ray training checkpoints

You download all resulting checkpoints from Ray training job.

In [ ]:
! gsutil -q cp -r {TRAINING_URI} {DELIVERABLES_PATH}

### Get the best training checkpoint

You use the `ExperimentAnalysis` to retrive the best checkpoint according to relevant metrics and mode.

In [ ]:
experiment_analysis = ExperimentAnalysis(DELIVERABLES_PATH)
log_path = experiment_analysis.get_best_logdir(metric="test_accuracy", mode="max")
best_checkpoint = experiment_analysis.get_best_checkpoint(
    log_path, metric="test_accuracy", mode="max"
)

### Get PyTorch Model from Ray TorchCheckpoint

Convert a TorchCheckpoint to Pytorch Model.

In [ ]:
torch_model = ray_torch.get_pytorch_model_from(best_checkpoint)

### Build PyTorch model archive (.mar) file

TorchServe allows you to serve Torch model by packaging all model artifacts into a single model archive file. In this case, the following information is required to create a standalone model archive:

1. Serialized file
2. Model file
3. Handler

#### Save the model

The `model.pt` contains the model state_dict.


In [ ]:
torch.save(torch_model.state_dict(), build_path / "model.pt")

#### Create the `model` module

The `model.py` file should contain the model architecture.


In [ ]:
model_script = """
import torch
import torch.nn as nn
import torch.nn.functional as F


class Cifar10Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
"""

with open(build_path / "model.py", "w") as model_file:
    model_file.write(model_script)
model_file.close()

#### Create the `custom_handler` module

The `custom_handler.py` file uses the TorchServe's inbuilt `image_classifier` handler name to handle custom TorchServe inference logic.

In [ ]:
custom_handler_script = '''
# Based on https://github.com/pytorch/serve/blob/master/examples/image_classifier/mnist/mnist_handler.py

from torchvision import transforms
from ts.torch_handler.image_classifier import ImageClassifier
from torch.profiler import ProfilerActivity


class Cifar10Classifier(ImageClassifier):
    """
    Cifar10Classifier handler class. This handler extends ImageClassifier class
    """

    label_names = [
        "plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"
    ]

    image_processing = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    def __init__(self):
        super(Cifar10Classifier, self).__init__()
        self.profiler_args = {
            "activities": [ProfilerActivity.CPU],
            "record_shapes": True,
        }

    def postprocess(self, data):
        """
        Post-process function to convert the predicted class id to label
        """
        predictions = data.argmax(1).tolist()
        return [self.label_names[pred] for pred in predictions]

'''

with open(build_path / "custom_handler.py", "w") as custom_handler_file:
    custom_handler_file.write(custom_handler_script)
custom_handler_file.close()

#### Package the model artifacts in a model archive file

Use the Torch Model Archiver tool to create a model archive file (.mar).


In [ ]:
build_script = """
torch-model-archiver -f --model-name cifar10 \
    --version 1.0  \
    --model-file model.py \
    --serialized-file model.pt \
    --handler custom_handler.py \
    --export-path .
"""

with open(build_path / "build.sh", "w") as build_file:
    build_file.write(build_script)
build_file.close()

In [ ]:
! cd {str(build_path)} && chmod +x ./build.sh && ./build.sh

#### Upload `mar` to bucket

Store the .mar file to Cloud bucket.

In [ ]:
!gsutil cp {str(build_path)}/cifar10.mar {BUILD_URI}/model.mar

### Register model in Vertex AI Model Registry

Register the model as a Model Resource in Vertex AI Model Registry.

In [ ]:
registered_model = vertex_ai.Model.upload(
    display_name=MODEL_NAME,
    serving_container_image_uri=DEPLOY_IMAGE_URI,
    artifact_uri=BUILD_URI,
)

### Deploy model for predictions

Create a Vertex AI Endpoint and deploy the registered model for predictions.

In [ ]:
endpoint = registered_model.deploy(
    deployed_model_display_name=MODEL_NAME,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=None,
    accelerator_count=0,
)

## Make online predictions

You sample an image from the `CIFAR10` dataset for getting online predictions.

In [ ]:
test_dataset = datasets.CIFAR10(
    root=tests_path / "data",
    transform=transforms.ToTensor(),
    train=False,
    download=True,
)

Visualize the sampled image.

In [ ]:
pil_image = plot_image_sample(test_dataset)

Run a prediction request and get the predicted class.

In [ ]:
predictions = predict_from_image(pil_image, endpoint)

for pred in predictions:
    print("Predicted class:", pred)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
import os

delete_endpoint = False
delete_model = False
delete_ray_cluster = False
delete_bucket = False
delete_tutorial = False

# Delete endpoint resource
if delete_endpoint or os.getenv("IS_TESTING"):
    endpoint.delete(force=True)

# Delete model resource
if delete_model or os.getenv("IS_TESTING"):
    registered_model.delete()

# Delete ray on vertex cluster
if delete_ray_cluster or os.getenv("IS_TESTING"):
    vertex_ray.delete_ray_cluster(ray_cluster.cluster_resource_name)

# Delete tutorial folder
if delete_tutorial or os.getenv("IS_TESTING"):
    shutil.rmtree(tutorial_path)

# Delete Cloud Storage objects that were created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -q -m rm -r $BUCKET_URI